# Sensitivity Analysis: OAT vs Variance-Based Methods

This notebook demonstrates sensitivity analysis techniques for rocket simulations, comparing:

1. **OAT (One-At-a-Time)**: Quick parameter screening method
2. **Variance-Based**: Rigorous statistical method following RocketPy standards

## Learning Objectives

- Understand when to use each sensitivity method
- Integrate Monte Carlo simulations with sensitivity analysis
- Interpret sensitivity coefficients and Linear Approximation Error (LAE)
- Identify critical parameters for rocket design

## Prerequisites

Run the Monte Carlo notebook (`02_monte_carlo_ensemble.ipynb`) first to generate simulation data.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.config_loader import ConfigLoader
from src.monte_carlo_runner import MonteCarloRunner
from src.variance_sensitivity import VarianceBasedSensitivityAnalyzer
from src.sensitivity_analyzer import OATSensitivityAnalyzer
from src.sensitivity_utils import (
    estimate_parameter_statistics,
    create_sensitivity_comparison,
    filter_significant_parameters
)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ All imports successful")

## Part 1: Theory - Understanding Sensitivity Methods

### OAT (One-At-a-Time) Method

**How it works:**
1. Run baseline simulation with nominal parameters
2. For each parameter, vary it by ±X% while keeping others fixed
3. Calculate sensitivity index from output changes

**Pros:**
- Simple and intuitive
- Shows directional effects (increase/decrease)
- Good for quick parameter screening

**Cons:**
- Doesn't quantify variance contribution
- Expensive: requires 2N+1 simulations
- No statistical validation

**Formula:**
```
Sensitivity Index = |((high - low) / 2 / baseline) / (variation_percent / 100)|
```

---

### Variance-Based Method (RocketPy Standard)

**How it works:**
1. Run Monte Carlo simulation with parameter uncertainties
2. Fit multiple linear regression: y = β₀ + β₁x₁ + β₂x₂ + ...
3. Calculate sensitivity coefficients from variance decomposition

**Pros:**
- Quantifies variance contribution (percentage)
- Statistical rigor with confidence intervals
- Efficient: reuses Monte Carlo data
- Validated with Linear Approximation Error (LAE)

**Cons:**
- Requires Monte Carlo simulation first
- Assumes linear relationships (check LAE)

**Formula:**
```
S(j) = 100 × (β_j² × σ_j²) / (σ_ε² + Σ_k(σ_k² × β_k²))

where:
- S(j) = sensitivity coefficient for parameter j
- β_j = regression coefficient
- σ_j = standard deviation of parameter j
- σ_ε = residual standard error
```

**Interpretation:**
- S(j) = 70% means: if parameter j were known perfectly, output variance would decrease by 70%
- LAE < 10%: Linear approximation is excellent
- LAE > 30%: Non-linear effects may be significant

## Part 2: Setup - Load Configuration and Run Monte Carlo

First, we need Monte Carlo simulation data for variance-based analysis.

In [ ]:
# Load configuration
config_path = Path('../configs/complete_example.yaml')

config_loader = ConfigLoader()
config_loader.load_from_yaml(str(config_path))

rocket_cfg = config_loader.get_rocket_config()
motor_cfg = config_loader.get_motor_config()
env_cfg = config_loader.get_environment_config()
sim_cfg = config_loader.get_simulation_config()

print(f"Configuration loaded: {rocket_cfg.name}")
print(f"Dry mass: {rocket_cfg.dry_mass_kg:.2f} kg")
print(f"Motor: {motor_cfg.thrust_source}")

In [ ]:
# Load configuration
config_path = Path('../configs/monte_carlo/01_basic_mc.yaml')

config_loader = ConfigLoader()
config_loader.load_from_yaml(str(config_path))

rocket_cfg = config_loader.get_rocket_config()
motor_cfg = config_loader.get_motor_config()
env_cfg = config_loader.get_environment_config()
sim_cfg = config_loader.get_simulation_config()

print(f"Configuration loaded: {rocket_cfg.name}")
print(f"Dry mass: {rocket_cfg.dry_mass_kg:.2f} kg")
print(f"Motor: {motor_cfg.thrust_source}")

In [ ]:
# Add parameter variations
# These represent measurement uncertainties and manufacturing tolerances

mc_runner.add_parameter_variation(
    parameter_path="rocket.dry_mass_kg",
    mean=rocket_cfg.dry_mass_kg,
    std=0.5,  # ±0.5 kg uncertainty
    distribution="normal"
)

mc_runner.add_parameter_variation(
    parameter_path="environment.wind.velocity_ms",
    mean=env_cfg.wind.velocity_ms,
    std=2.0,  # ±2 m/s wind variation
    distribution="normal"
)

mc_runner.add_parameter_variation(
    parameter_path="rocket.fins.root_chord_m",
    mean=rocket_cfg.fins.root_chord_m,
    std=0.005,  # ±5mm manufacturing tolerance
    distribution="normal"
)

print("Parameter variations configured:")
for param, config in mc_runner.parameter_variations.items():
    print(f"  {param}: μ={config['mean']:.3f}, σ={config['std']:.3f}")

In [ ]:
# Run Monte Carlo simulation
print("Running Monte Carlo simulation (this may take a few minutes)...")

results = mc_runner.run(parallel=True, max_workers=4)

print(f"\nSimulation complete!")
print(f"Successful: {len(results)}/{mc_runner.num_simulations}")
print(f"Failed: {len(mc_runner.failed_simulations)}")

# Show statistics
mc_runner.print_statistics_summary()

## Part 3: Variance-Based Sensitivity Analysis

Now we perform variance-based sensitivity analysis on the Monte Carlo results.

In [ ]:
# Export Monte Carlo data for sensitivity analysis
parameters_df, targets_df = mc_runner.export_for_sensitivity(
    parameter_names=list(mc_runner.parameter_variations.keys()),
    target_names=["apogee_m", "max_velocity_ms", "lateral_distance_m"]
)

print(f"Exported {len(parameters_df)} samples")
print(f"Parameters: {list(parameters_df.columns)}")
print(f"Targets: {list(targets_df.columns)}")

# Preview data
print("\nParameter samples (first 5):")
display(parameters_df.head())

print("\nTarget variables (first 5):")
display(targets_df.head())

In [ ]:
# Create variance-based sensitivity analyzer
analyzer = VarianceBasedSensitivityAnalyzer(
    parameter_names=list(parameters_df.columns),
    target_names=list(targets_df.columns)
)

# Set nominal parameters from Monte Carlo configuration
metadata = mc_runner.get_parameter_metadata()
means = {param: meta['mean'] for param, meta in metadata.items()}
stds = {param: meta['std'] for param, meta in metadata.items()}

analyzer.set_nominal_parameters(means=means, stds=stds)

print("Analyzer configured with nominal parameters:")
for param in parameters_df.columns:
    print(f"  {param}: μ={means[param]:.3f}, σ={stds[param]:.3f}")

In [ ]:
# Fit regression models
print("Fitting regression models...\n")
analyzer.fit(parameters_df, targets_df)

print("✓ Regression models fitted successfully!")

In [ ]:
# Print comprehensive sensitivity summary
analyzer.print_summary()

### Interpreting the Results

The sensitivity table shows:

1. **Sensitivity(%)**: Percentage contribution to output variance
   - If you could measure this parameter perfectly (no uncertainty), how much would the output variance decrease?
   - Example: S = 70% means 70% of output variance is due to this parameter's uncertainty

2. **Nominal mean/sd**: The mean and standard deviation used in the analysis

3. **Effect per sd**: Change in output for one standard deviation change in parameter
   - Units: [output units] per [input standard deviation]
   - Shows practical impact magnitude

4. **Linear Approximation Error (LAE)**:
   - LAE < 10%: Linear model is excellent ✓
   - LAE < 30%: Linear model is adequate ~
   - LAE > 30%: Non-linear effects significant ✗

**Key Insight**: Focus on parameters with sensitivity > LAE. Parameters with sensitivity < LAE may not be statistically significant.

In [ ]:
# Get importance ranking for apogee
ranking_apogee = analyzer.get_importance_ranking("apogee_m")

print("Parameter Importance Ranking for Apogee:")
print("=" * 50)
for i, (param, sensitivity) in enumerate(ranking_apogee, 1):
    print(f"{i}. {param:<35} {sensitivity:>6.2f}%")

# Identify significant parameters
lae_apogee = analyzer.lae["apogee_m"]
sensitivities = analyzer.sensitivity_coefficients["apogee_m"]

significant_params = filter_significant_parameters(sensitivities, lae_apogee)

print(f"\nSignificant parameters (sensitivity > LAE={lae_apogee:.2f}%):")
for param in significant_params:
    print(f"  ✓ {param}: {sensitivities[param]:.2f}%")

In [ ]:
# Generate sensitivity bar plots for all targets
output_dir = Path('../outputs/sensitivity')
output_dir.mkdir(parents=True, exist_ok=True)

for target_name in targets_df.columns:
    plot_path = output_dir / f"sensitivity_bars_{target_name}.png"
    
    fig = analyzer.plot_sensitivity_bars(
        output_path=str(plot_path),
        target_name=target_name,
        figsize=(10, 6)
    )
    
    print(f"Saved plot: {plot_path}")
    
    # Display inline
    plt.show()

## Part 4: OAT Sensitivity Analysis (for Comparison)

Now let's run OAT analysis on the same parameters to compare methods.

In [ ]:
# Create OAT analyzer
oat_analyzer = OATSensitivityAnalyzer(
    base_rocket_config=rocket_cfg,
    base_motor_config=motor_cfg,
    base_environment_config=env_cfg,
    base_simulation_config=sim_cfg
)

# Add same parameters with 10% variation
for param in parameters_df.columns:
    oat_analyzer.add_parameter(
        parameter_path=param,
        variation_percent=10.0
    )

print("OAT analyzer configured")

In [ ]:
# Run OAT analysis for apogee
print("Running OAT sensitivity analysis...\n")
oat_analyzer.run(output_metric="apogee_m")

# Print summary
oat_analyzer.print_summary()

In [ ]:
# Generate OAT tornado diagram
tornado_path = output_dir / "tornado_diagram_oat.png"

oat_analyzer.plot_tornado_diagram(
    output_path=str(tornado_path),
    title="OAT Sensitivity: Apogee (±10% variation)"
)

print(f"Saved tornado diagram: {tornado_path}")

# Display
img = plt.imread(tornado_path)
plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis('off')
plt.show()

## Part 5: Comparing OAT vs Variance-Based Methods

Let's compare the results from both methods.

In [ ]:
# Get OAT ranking
oat_ranking = oat_analyzer.get_importance_ranking()

print("Method Comparison for Apogee")
print("=" * 70)
print(f"{'Parameter':<35} {'Variance-Based':<20} {'OAT Index':<15}")
print("=" * 70)

for param in parameters_df.columns:
    var_sens = sensitivities.get(param, 0)
    oat_sens = oat_analyzer.sensitivity_results.get(param, {}).get('sensitivity_index', 0)
    
    print(f"{param:<35} {var_sens:>8.2f}% {' '*10} {oat_sens:>8.3f}")

print("=" * 70)
print(f"LAE: {lae_apogee:.2f}%")
print("\nNote: Variance-based values are percentages of variance contribution.")
print("OAT values are dimensionless sensitivity indices.")

### Key Differences

| Aspect | Variance-Based | OAT |
|--------|---------------|-----|
| **Metric** | Variance contribution (%) | Sensitivity index (dimensionless) |
| **Interpretation** | "How much variance" | "How sensitive" |
| **Simulations** | Reuses MC data (efficient) | 2N+1 new runs |
| **Statistical rigor** | Yes (LAE validation) | No |
| **Ranking** | Usually similar | Usually similar |
| **Best for** | Quantitative analysis, publications | Quick screening |

**When rankings differ:** Check LAE. High LAE suggests non-linear effects that OAT may capture better.

## Part 6: Practical Insights - Design Recommendations

Based on sensitivity analysis, we can make design recommendations.

In [ ]:
# Calculate potential variance reduction
from sensitivity_utils import calculate_variance_reduction

# Scenario 1: Control top parameter only
top_param = significant_params[0] if significant_params else ranking_apogee[0][0]
reduction_top = calculate_variance_reduction(
    sensitivities,
    [top_param]
)

print("Design Improvement Scenarios:")
print("=" * 60)
print(f"\nScenario 1: Control {top_param}")
print(f"  Variance reduction: {reduction_top:.1f}%")
print(f"  Action: Improve measurement/manufacturing tolerance")

# Scenario 2: Control top 2 parameters
if len(significant_params) >= 2:
    top_2_params = significant_params[:2]
    reduction_top2 = calculate_variance_reduction(sensitivities, top_2_params)
    
    print(f"\nScenario 2: Control {' and '.join(top_2_params)}")
    print(f"  Variance reduction: {reduction_top2:.1f}%")
    print(f"  Action: Focus quality control on these two parameters")

# Scenario 3: Control all significant parameters
if significant_params:
    reduction_all = calculate_variance_reduction(sensitivities, significant_params)
    
    print(f"\nScenario 3: Control all significant parameters")
    print(f"  Parameters: {len(significant_params)}")
    print(f"  Variance reduction: {reduction_all:.1f}%")
    print(f"  Remaining uncertainty: {100 - reduction_all:.1f}% (from non-significant params + LAE)")

### Design Recommendations

Based on the sensitivity analysis:

1. **Focus Quality Control**: Prioritize measurement/manufacturing tolerance for high-sensitivity parameters

2. **Cost-Benefit Analysis**: 
   - Tightening tolerance on low-sensitivity parameters has minimal impact
   - Investment should match sensitivity ranking

3. **Flight Predictions**:
   - Prediction uncertainty is dominated by high-sensitivity parameters
   - Better measurements of these parameters will improve prediction accuracy

4. **Design Margins**:
   - Account for uncertainty propagation from significant parameters
   - Use prediction intervals for safety margins

## Part 7: Save Results for Future Use

In [ ]:
# Save Monte Carlo data in RocketPy format
mc_output_dir = output_dir / 'monte_carlo_data'

input_path, output_path = mc_runner.save_rocketpy_format(
    output_dir=str(mc_output_dir),
    filename_prefix="sensitivity_example"
)

print(f"Monte Carlo data saved:")
print(f"  Input:  {input_path}")
print(f"  Output: {output_path}")

# Export sensitivity results
from sensitivity_utils import export_sensitivity_to_json

json_path = output_dir / 'sensitivity_results.json'

export_sensitivity_to_json(
    sensitivity_coefficients=analyzer.sensitivity_coefficients,
    lae=analyzer.lae,
    nominal_parameters=metadata,
    output_file=json_path
)

print(f"\nSensitivity results saved: {json_path}")

## Summary and Next Steps

### What We Learned

1. **Variance-based sensitivity** provides quantitative variance contribution with statistical validation
2. **OAT sensitivity** is useful for quick screening and understanding directional effects
3. **LAE** validates the linear approximation assumption
4. **Sensitivity ranking** guides design priorities and quality control focus

### Recommended Workflow

```
1. Monte Carlo simulation (100+ samples)
   ↓
2. Variance-based sensitivity analysis
   ↓
3. Check LAE:
   - LAE < 10%: Results are reliable ✓
   - LAE > 30%: Consider non-linear effects or more samples
   ↓
4. Focus on parameters with sensitivity > LAE
   ↓
5. Design improvements based on variance reduction potential
```

### Further Reading

- RocketPy Documentation: [Sensitivity Analysis](https://docs.rocketpy.org/)
- Sobol' variance decomposition theory
- Morris screening method for large parameter spaces

### Try It Yourself

1. Modify parameter uncertainties and re-run
2. Add more parameters to the analysis
3. Analyze different target variables (max velocity, landing distance)
4. Compare results with different rocket configurations